# Notes`
adapted from https://colab.research.google.com/drive/1LliVq9wD4N4smXA0aST-quMmBFdFSdxz#scrollTo=mCKxsaFdwYZ4

In [ ]:
import os

In [ ]:
os.getcwd()
os.listdir()

In [ ]:
my_vcf_gz_file = '15.27999021-28345461.ALL.chr15.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'
bytes_to_mb = 1000000
os.path.getsize(my_vcf_gz_file)/bytes_to_mb

Source scripts with functions in it

In [ ]:
run "vcf_function_definitions.py"

### Process VCF file

In [ ]:
# convert a .vcf.gz file to a a .vcf file
## all .gz files are compressed
## .vcf are a specially laid out text file

vcfgz_to_gz(vcf_gz_file = my_vcf_gz_file,
            vcf_name = "output.vcf")

In [ ]:
os.listdir()

## Convert VCF file to dataframe

Data from 1000 Genomes are in phased format, which represents information on the chromosome that an allele originates from (paternal or maternal).  This information has many important uses, but not for what we're doing.  

The time it takes to do this varies a lot.  ACKR1 is a small gene and takes less than 1 second.  OCA2 is a large gene and takes about 20 seconds.

In [ ]:
# vcf_file: your .vcf file
# csv_file: name of .csv file you want created
phased_snps = vcf_to_df(vcf_file = "output.vcf",
                        csv_file = "output_phased_snps.csv")

In [ ]:
os.listdir()

## Look at dataframe

In [ ]:
phased_snps.head()

In [ ]:
phased_snps.tail()

# ☯ Run Quality Control & Examine your SNPs!

In [ ]:
phased_snps.columns

In [ ]:
len(phased_snps.columns)

We can look at the first 10 column names to see what they show.

In [ ]:
print(list(phased_snps.columns[0:10]))

These should be familiar from the preview panel of Data Slicer.

The `.shape` attribute of a pandas datafame tells us the rows (first number) and columns (second number)

In [ ]:
phased_snps.shape

We can call `.head()` on the dataframe and look at things.

In [ ]:
phased_snps.head()

In [ ]:
phased_snps.tail()

## Columns of a vcf file

Columns we will use in this class are <u>underlined.</u>

* **<u>CHROM</u>** = **chromosome**
* **<u>POS</u>** = **genomic position** on chromosome
* **<u>ID</u>** = **rs code / rs ID** for SNP
* **<u>REF</u>** = the **most common allele**; typically what is in the reference genome
* **<u>ALT</u>** = **alternative allele** OR **alleles**.  Usually there is only 1 alternative allele, BUT there can be moer!
* **QUAL** = **quality score**
* **FILTER** = related to quality control
* **INFO** = **meta-data** for each SNP
* **FORMAT** = format of actual SNP data; "GT" for "genotype" for us

Individuals have ID codes like "**HG00096**" and "**NA21129**".

The population and sex of each person is known, and if they have any children or parents also in the dataset, but this data is stored in a different file.

# On saving lots of files and reloading them

Throughout this workflow I will be

1. Saving backup copies of key files (and downloading them)
1. Re-loading the backup copy when I move on to the next step

I do this to

1. Save my work in case I break the code later
1. So I don't have to re-run time consuming steps
1. Make the code modular for moving around to new projects.

## Featurize SNPs

In [ ]:
os.listdir()

In [ ]:
# load snps
import pandas as pd
csv_file = "output_phased_snps.csv"
phased_snps = pd.read_csv(csv_file)

Look at the data

In [ ]:
phased_snps.shape

In [ ]:
phased_snps.head()

Save names of colmns to an object

In [ ]:
columns = phased_snps.columns
columns

The first 10 columns are metadata

In [ ]:
columns[0:9]

The 11th column is the first person's data

In [ ]:
column11 = columns[10]

Look at the columns

In [ ]:
phased_snps[column11]
phased_snps[columns[10]]

## Isolating mono-nucleotide, bi-allelic SNPS

MOST variant in 1000 genomes are

1. mono-nucleotides
1. bi-allelic

Mononucleotides means just 1 nucleotides: A, T, C, or G.  Bi-allelic means 2 possible alleles This means that they just have single-nucleotide transition or transversion point mutations with just 2 variants.  That is, some people in the world have e.g. a T at a specific genomic address, and others have a C.  

But, some SNPs are polymorphic - tri- or tetra-allelic: e.g, for tri-allelic, e.g  some people have A, some have C, and some have T. For a tri-allelic SNP, there will be one allele listed for the REF, and two for the ALT.  For tetra allelic one will be listed as REF, and 3 for the alt.

Also, 1000 Genomes data also includes insertions and deletions, e.g. some people are A and others don't have an A.  Finally, sometimes the variant is a dinucleotide (e.g. AA, AT, TA) or larger sequence, such as GCGCGGC.  

For most analyses we want just basic mono-nucleotide, bi-allelic SNPS.  That is, we want only A, T, C, and G as our REF and ALT alleles, an we want just one nucleotide listed as the REF, and one as an ALT.


In the code below we will explore the types of variants we have and then remove everything but bi-allelic SNPs.

For related background info, see [here](https://docs.google.com/document/d/1Lgc5vVsAKyYJsCAf50Cb_T1uZbfU8TbUbUgpImRwzww/edit#bookmark=id.oyw4n2k69x05) in the outline/notes for the unit.



The `.value_counts()` method counts up all the possible values that appear in a column.

In [ ]:
# these commands both do the same thing
phased_snps.REF.value_counts()
phased_snps["REF"].value_counts()

In [ ]:
# these commands both do the same thing
phased_snps.ALT.value_counts()
phased_snps["ALT"].value_counts()

Save info on these possibilities


In [ ]:
var_counts_all_REF = phased_snps.REF.value_counts()
var_counts_all_ALT = phased_snps.ALT.value_counts()

In [ ]:
import seaborn as sns
var_counts_all_REF = pd.Series(var_counts_all_REF)
var_counts_all_ALT = pd.Series(var_counts_all_ALT)


In [ ]:
var_counts_all_REF

In [ ]:
sns.countplot(phased_snps, y="REF");

I can see the 1 long reference allele that shows up using pandas column selection code.  (🙈You don't need to know how to do this on your own)

In [ ]:
# 🙈
phased_snps[phased_snps.REF == "GA"]

The referene genome has "TAACA" and the ALT is "T", implying a deletion of AACA.  Further anlysis would be required to determine, however, which one is the ancestral allele.  1000 Genomes list the allele the reference genome, NOT the most common allele globablly, so its possible that "T" is more common than TAACA.

We can select what we want using the `.isin()` method of pandas dataframe. (🙈 You don't need to know how this function works)

This gives us a TRUE or FALSE logical answer of whether an element is in a list.



In [ ]:
#🌊
# 🙈
real_SNPs = ["A","T","C","G"]
onlyATCG_REF = phased_snps.REF.isin(real_SNPs)

# THIS NEED TO BE FIXED
onlyATCG_ALT = phased_snps.ALT.isin(real_SNPs)

print(onlyATCG_REF[1:10])



We can now use this vector to isolated just the biallelic SNPS into a new dataframe

### THIS NEEDS TO BE FIXED - need to re-run isin after first processing

In [ ]:
phased_snps_biallelic = phased_snps[onlyATCG_REF]
phased_snps_biallelic = phased_snps_biallelic[onlyATCG_ALT]

In [ ]:
phased_snps.head()

The size of our original dataframe

In [ ]:
phased_snps.shape

In [ ]:
phased_snps_biallelic.shape

In [ ]:
phased_snps_biallelic.REF.value_counts()

In [ ]:
phased_snps_biallelic.ALT.value_counts()

## Removing metadata columns

For most of our purposes we won't need the meta data columns



In [ ]:
phased_snps_biallelic.head()

In [ ]:
phased_snps_biallelic.columns[0:9]


In [ ]:
phased_snps_biallelic.columns[10]

In [ ]:
 phased_snps_biallelic.ID

In [ ]:
phased_snps_biallelic.POS

Before we do this we want to set the rsID to be the column index

# WANRING - rs IDS arean't always present!!!!

In [ ]:
rsID = phased_snps_biallelic.ID   # ACHTUNG  - doesn't always show up!
POS  = phased_snps_biallelic.POS
phased_snps_biallelic.index = POS

In [ ]:
phased_snps_biallelic.head()

We'll make a copy of the dataframe using the `.copy()` method then remove the first 9 columns

In [ ]:
# copy the dtaframe
phased_snps_biallelic2 = phased_snps_biallelic.copy()

In [ ]:
meta_columns = phased_snps_biallelic2.columns[0:9]

In [ ]:
meta_columns

We can then remove the specified columns using the `.drop()` method

In [ ]:
phased_snps_biallelic2= phased_snps_biallelic2.drop(meta_columns,
                                                          axis=1)


In [ ]:
phased_snps_biallelic.shape

In [ ]:
phased_snps_biallelic2.shape

In [ ]:
phased_snps_biallelic2.head()

In [ ]:
phased_snps_biallelic2.columns[0:10]

## Featurize SNPs

This takes some time ⌛

We want to convert the phased snps (e.g 0|0, 0|1, 1|1 etc) into unphased, numeric features.


This is the code I'm using to do this.  I've also written a function to do it : )

In [ ]:
#🐼
df_shape = phased_snps_biallelic2.shape
df_nrows = df_shape[0]
df_ncols = df_shape[1]
df_cols= phased_snps_biallelic2.columns
df_working = phased_snps_biallelic2
for i in range(0,df_ncols):
    if i == 1000:
        print("1000 individuals processed, 1500 to go : )")
    if i == 2000:
        print("1000 individuals processed, 500 to go : )")
    df_working[df_cols[i]][df_working[df_cols[i]] == "0|0"] = 0
    df_working[df_cols[i]][df_working[df_cols[i]] == "1|1"] = 2
    df_working[df_cols[i]][df_working[df_cols[i]] == "1|0"] = 1
    df_working[df_cols[i]][df_working[df_cols[i]] == "0|1"] = 1
my_snps012 = df_working

Use the function

In [ ]:
featurized_snps = phased_to_featurized(phased_snps_biallelic2)

In [ ]:
featurized_snps.head()

In [ ]:
featurized_snps.columns

In [ ]:
featurized_snps.HG00096.value_counts()


# Transpose

VCF files have SNPs in rows and people in columns.

For most analyes we want this flipped: people in rows and SNPs as columns (features).

This flipping can be done with the pandas .tranpose() method.

In [ ]:
featurized_snps.head()

In [ ]:
#🌈
featurized_snps_T = featurized_snps.transpose()

In [ ]:
featurized_snps_T.head()

In [ ]:
featurized_snps_T.head()

In [ ]:
featurized_snps_T.columns

In [ ]:
featurized_snps.shape

In [ ]:
featurized_snps_T.shape

# Save the file now!!!

This file is ready for analysis - SAVE IT!  THEN DOWNLOAD IT!



In [ ]:
csv_file = "snps_featurized.csv"
featurized_snps_T.to_csv(csv_file,
               index=False)

In [ ]:
os.listdir()

# Drop NAs

NA stands or "Not available".  These are values that are missing, weren't entered, etc.  

Sometimes you see "NaN", which means "Not a number."

NAs can be dropped "row wise" - which removes a row if ANY value in the row is NA.  NAs can be dropped "column wise", which drops an entire column if there is even 1 NA in it.

NAs are rare in 1000 genomes data, but common elsewhere.


In [ ]:
#row-wise deletion -
featurized_snps_noNA1 = featurized_snps_T.dropna()

# column-wise deletion
featurized_snps_noNA2 = featurized_snps_T.dropna(axis = 1)

In this case, no NAs were dropped.

In [ ]:
featurized_snps_T.shape

In [ ]:
featurized_snps_noNA1.shape

In [ ]:
featurized_snps_noNA2.shape

# Drop invariant loci

Its possible that in the raw data of or through our processing its ended up that all peole have the exact same value for a SNPs.  This will cause problems for various reasons, so we want to drop these invariant loci.

In [ ]:
featurized_snps_noNA1.tail()

## AD HOC SOLUTION - last column has issues
just dropping to keep my life simple for now

In [ ]:
featurized_snps_noNA1.drop(featurized_snps_noNA1.tail(1).index,inplace=True)

In [ ]:
featurized_snps_noNA1.tail()

In [ ]:
stds = featurized_snps_noNA1.std()


In [ ]:
stds_0 = stds == 0

In [ ]:
stds_0

In [ ]:
sum(stds_0)

In [ ]:
featurized_snps_noNA1.columns[stds_0]

In [ ]:
featurized_snps_noNA1.columns[-stds_0]

In [ ]:
len(featurized_snps_noNA1.columns)

In [ ]:
featurized_snps_noNA1.columns

In [ ]:
featurized_snps_no_invar = featurized_snps_noNA1[featurized_snps_noNA1.columns[-stds_0]]

# Center data

The middle of a dataset is defined by the mean (or the median, but that won't work for us for various reasons).

For more various reasons, most machine learning algorithms and many advanced stats methods like to have their data centered around zero.

This means that the data are moved by subtracting the mean from each value.

I can get the mean of each column (SNP) using the .mean() method

In [ ]:
featurized_snps_no_invar.mean()

In [ ]:
snp_means = featurized_snps_no_invar.mean()
featurized_snps_center = featurized_snps_no_invar - snp_means

In [ ]:
featurized_snps_center.mean()

In [ ]:
featurized_snps_center.tail()

# Scale data by SD

For more various reasons, we often "scale" our data by the standard deviation or some other function.

This means we take the data and divide by the standard deviation


In [ ]:
snp_std = featurized_snps_center.std()

In [ ]:
snp_std

In [ ]:
snp_std.min()

In [ ]:
featurized_snps_center_scale = featurized_snps_center/snp_std

In [ ]:
featurized_snps_center_scale.mean()

In [ ]:
featurized_snps_center_scale.std()

In [ ]:
featurized_snps_center_scale.to_csv("snps_for_analysis.csv")

# The data



In [ ]:
featurized_snps_center_scale.head()

In [ ]:
!wget https://raw.githubusercontent.com/brouwern/compbio2023/main/1000genomes_people_info.csv

In [ ]:
!ls

In [ ]:
import pandas as pd

people_file = "1000genomes_people_info.csv"
people = pd.read_csv(people_file)

In [ ]:
people.head()

In [ ]:
people = people.set_index("sample")

In [ ]:
people.head()

In [ ]:
people.super_pop.value_counts()

In [ ]:
featurized_snps_center_scale.head()

In [ ]:
featurized_snps_center_scale.index.names = ["sample"]

In [ ]:
featurized_snps_center_scale.head()

In [ ]:
featurized_snps_center_scale.head()

In [ ]:
people_columns = ["pop","super_pop"]
snps = pd.merge(people[people_columns],
                featurized_snps_center_scale,
                left_index=True, right_index=True)

In [ ]:
featurized_snps_center_scale.shape

In [ ]:
snps.shape

In [ ]:
snps.head()

Gambian in Western Division – Mandinka [GWD]
Japanese in Tokyo, Japan [JPT]
Finnish in Finland [FIN]



In [ ]:
snps["pop"]

# Distances

In [ ]:
snps.head()

In [ ]:
snps = snps.set_index(["pop","super_pop"])

In [ ]:
snps.head()

In [ ]:
snps_super_pop_mean = snps.groupby('super_pop').mean()

In [ ]:
snps_pop_mean = snps.groupby(["super_pop","pop"]).mean()

In [ ]:
snps_pop_mean

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

pairwise_distr_pop = pairwise_distances(snps_pop_mean, metric='euclidean')

In [ ]:
pairwise_dist_super_pop = pairwise_distances(snps_super_pop_mean, metric='euclidean')

In [ ]:
pairwise_dist_super_pop

In [ ]:
dist_mat_super_pop = pd.DataFrame(data=pairwise_dist_super_pop)

Africans (AFR), Admixed Americans (AMR), East Asians (EAS), Europeans (EUR) and South Asians (SAS)

In [ ]:
dist_mat_super_pop.columns = ["AFR","EAS","EUR","SAS","AMR"]

In [ ]:
dist_mat_super_pop.insert(0, "super_pop", ["AFR","EAS","EUR","SAS","AMR"], True)

In [ ]:
dist_mat_super_pop

In [ ]:
dist_mat_super_pop = dist_mat_super_pop.set_index("super_pop")

In [ ]:
dist_mat_super_pop

In [ ]:
dist_mat_super_pop.to_csv(csv_file,
               index=True)

# Create full matrix

In [ ]:
snps_no_meta = snps.drop(["pop","super_pop"], axis = 1)

In [ ]:
snps_no_meta.head()

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

pairwise_dist = pairwise_distances(snps_no_meta, metric='euclidean')

In [ ]:
pairwise_dist

# Further explorations

In [ ]:
focal_pops = snps["pop"].isin(['GWD', 'JPT',"FIN"])

In [ ]:
snps_focal_pops = snps[focal_pops]

In [ ]:
snps_focal_pops.shape

In [ ]:
snps_focal_pops_rand = snps_focal_pops.sample(n=30, random_state=1)
snps_focal_pops_rand["pop"].value_counts()

In [ ]:
snps_focal_pops_rand2 = snps_focal_pops_rand.drop(["pop","super_pop"], axis = 1)

snps_focal_pops_rand2.head()

In [ ]:
import numpy as np
from statsmodels.multivariate.pca import PCA
pc = PCA(snps_focal_pops_rand2)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(snps_focal_pops_rand2)

plt.scatter(x, y, c=kmeans.labels_)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib as mp


linkage_data = linkage(snps_focal_pops_rand2,
                       method='complete',
                       metric='euclidean')




In [ ]:
dendrogram(linkage_data,labels = list(snps_focal_pops_rand.super_pop))

plt.show()
mp.rcParams['figure.figsize'] = [14, 8]


# Allele frequencies

In [ ]:
#🌋
import numpy as np
n_cols = my_snps012_T.shape[1]
Af_vect = list(np.full(shape=n_cols, fill_value=0))
af_vect = list(np.full(shape=n_cols, fill_value=0))
columns = my_snps012_T.columns
for i in range(0, n_cols):
    col_i = columns[i]

    col_working = my_snps012_T[col_i]

    geno_freq_i = col_working.value_counts()

    AA = 0
    Aa = 0
    aa = 0

    AA = list(geno_freq_i[geno_freq_i.index == 0])
    Aa = list(geno_freq_i[geno_freq_i.index == 1])
    aa = list(geno_freq_i[geno_freq_i.index == 2])

    if len(AA) == 0:
        AA = [0]
    if len(Aa) == 0:
        Aa = [0]
    if len(aa) == 0:
        aa = [0]

    A = 2*AA[0] + 1*Aa[0]
    a = 1*Aa[0] + 2*aa[0]
    N = A+a
    Af = A/N
    Af_vect[i] = Af

In [ ]:
for i in range(0,len(Af_vect)):
    af_vect[i] = 1 - Af_vect[i]

In [ ]:
import seaborn as sns
Af_ser = pd.Series(Af_vect)
af_ser = pd.Series(af_vect)
rs = my_snps012_T.columns

df = pd.DataFrame({"rs": rs,
                   "Af": Af_ser,
                   "af": af_ser})

In [ ]:
df[df.af.between(0.25, 0.35, inclusive=False)]

In [ ]:
df_rank = df.Af.rank()

In [ ]:
df = df.set_index(df_rank)

In [ ]:
df = df.sort_index()

In [ ]:
df[["rs","af"]].iloc[1:20]

In [ ]:
Af_ser_rank.max()
Af_ser_rank.min()

In [ ]:
Af_ser.iloc[Af_ser_rank]

In [ ]:
Af_vect[0:10]

In [ ]:
my_snps012_T["rs77368867"].value_counts()

In [ ]:
33*1 + 1*2

In [ ]:
(33*1 + 1*2)/(2470*2 + 33*2 + 1*2)

In [ ]:
n_ind = my_snps012_T.shape[0]
n_chrom = n_ind*2
n_chrom


In [ ]:
q = my_snps012_T["rs77368867"].sum()/n_chrom
q

In [ ]:
n_ind = len(my_snps012_T["rs56055582"])
n_chrom = n_ind*2
p = 1- my_snps012_T["rs56055582"].sum()/n_chrom
q = my_snps012_T["rs56055582"].sum()/n_chrom

In [ ]:
print(p)
print(q)

In [ ]:
AA = (p**2)*n_ind
Aa = 2*p*q*n_ind
aa = (q**2)*n_ind

AA + Aa + aa